BUAI 446 Assignment 1 - Akron Zoo  
Name: Ruihuang Yang  
NetID: rxy216  
Date: 09/07/2025  

### Import libraries

In [ ]:
import pandas as pd

### Load and Explore Data

In [2]:
# Load training and test datasets
train_data = pd.read_csv('data/ZOOLOG1-TRAIN-2025.csv')
test_data = pd.read_csv('data/ZOOLOG1-TEST-2025.csv')

print("Training Data Shape:", train_data.shape)
print("Test Data Shape:", test_data.shape)

Training Data Shape: (740, 20)
Test Data Shape: (303, 20)


### Basic Data Exploration

In [3]:
# Display first few rows of training data
print("Training Data - First 5 rows:")
print(train_data.head())

Training Data - First 5 rows:
   NID  UPD  benefits   costs   value  identity    know     sat     fle  \
0    1    0    0.8011 -0.2225 -0.2900    0.7353  0.4332  0.8511 -0.6795   
1    2    0   -0.8915 -0.2831  0.8196   -0.9243 -2.4690  0.8511 -0.6795   
2    3    1   -0.4484  1.0290 -0.4391   -0.8858 -0.2342  0.8511  0.8696   
3    4    0   -0.9444  0.4703 -1.6980   -0.2679 -0.6842 -0.5246 -1.2470   
4    5    0   -0.1004  0.8337 -0.4391   -1.0960 -0.6842 -0.5246  0.3787   

   trustfor  age  gender  educ  mstat  size  child1  dist  tvis  age_rec  \
0 -0.057583    4       2     1      1     3       4     3     3        3   
1  0.105467    2       2     3      1     5       3     4     2        1   
2 -0.057583    2       2     4      1     5       3     3     3        1   
3 -0.709000    4       1     3      1     5       3     4     3        3   
4 -1.732333    4       1     2      1     2       4     3     5        3   

   educnew  
0        0  
1        1  
2        2  
3        1

In [4]:
# Display column information
print("\nTraining Data - Column Info:")
print(train_data.info())


Training Data - Column Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NID       740 non-null    int64  
 1   UPD       740 non-null    int64  
 2   benefits  740 non-null    float64
 3   costs     740 non-null    float64
 4   value     740 non-null    float64
 5   identity  740 non-null    float64
 6   know      740 non-null    float64
 7   sat       740 non-null    float64
 8   fle       740 non-null    float64
 9   trustfor  740 non-null    float64
 10  age       740 non-null    int64  
 11  gender    740 non-null    int64  
 12  educ      740 non-null    int64  
 13  mstat     740 non-null    int64  
 14  size      740 non-null    int64  
 15  child1    740 non-null    int64  
 16  dist      740 non-null    int64  
 17  tvis      740 non-null    int64  
 18  age_rec   740 non-null    int64  
 19  educnew   740 non-null    int64  
dtypes:

In [5]:
# Display basic statistics
print("\nTraining Data - Descriptive Statistics:")
print(train_data.describe())


Training Data - Descriptive Statistics:
               NID         UPD    benefits       costs       value  \
count   740.000000  740.000000  740.000000  740.000000  740.000000   
mean    524.189189    0.500000   -0.032099   -0.007556   -0.035639   
std     305.265029    0.500338    1.209893    1.090441    1.053612   
min       1.000000    0.000000   -4.988000   -1.332000   -4.728000   
25%     257.500000    0.000000   -0.643500   -1.039000   -0.439100   
50%     531.500000    0.500000    0.152200    0.043310    0.370100   
75%     790.500000    1.000000    0.811500    0.673700    0.819600   
max    1042.000000    1.000000    1.355000    4.169000    0.819600   

         identity        know         sat         fle    trustfor         age  \
count  740.000000  740.000000  740.000000  740.000000  740.000000  740.000000   
mean    -0.025827   -0.020556   -0.027073   -0.006966    0.003556    3.459459   
std      1.105308    1.065872    1.037607    1.030317    0.884611    1.007270   
min 

In [6]:
# Check for missing values
print("\nMissing values in training data:")
print(train_data.isnull().sum())

print("\nMissing values in test data:")
print(test_data.isnull().sum())


Missing values in training data:
NID         0
UPD         0
benefits    0
costs       0
value       0
identity    0
know        0
sat         0
fle         0
trustfor    0
age         0
gender      0
educ        0
mstat       0
size        0
child1      0
dist        0
tvis        0
age_rec     0
educnew     0
dtype: int64

Missing values in test data:
NID         0
UPD         0
benefits    0
costs       0
value       0
identity    0
know        0
sat         0
fle         0
trustfor    0
age         0
gender      0
educ        0
mstat       0
size        0
child1      0
dist        0
tvis        0
age_rec     0
educnew     0
dtype: int64


In [7]:
# Check target variable distribution
print("\nTarget variable (UPD) distribution in training data:")
print(train_data['UPD'].value_counts())
print(f"\nUpgrade rate: {train_data['UPD'].mean():.3f}")


Target variable (UPD) distribution in training data:
UPD
0    370
1    370
Name: count, dtype: int64

Upgrade rate: 0.500
